# Pipeline for High-z Radio Galaxies 12: Create lists (files) with radio AGN candidates

## Introduction

In this file, three models will be applied consecutively in order to predict  
the detection of Radio Galaxies (radio AGN) and their redshift.  

In principle, this pipeline should be applied to data in Stripe 82. But  
it can be used with any other suitable dataset.

In [1]:
%matplotlib inline
# Static plots
#%matplotlib ipympl
# Interactive plots
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib import ticker
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patheffects as mpe
import matplotlib.patches as mpatches
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from astropy.visualization import LogStretch, PowerStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay
import sklearn.pipeline
import colorcet as cc
from pycaret import classification as pyc
from pycaret import regression as pyr
from joblib import dump, load
import pandas as pd
import mpl_scatter_density
import global_variables as gv
import global_functions as gf

In [2]:
mpl.rcdefaults()

---

## Reading data

Flags.

In [3]:
save_plot_flag      = False
save_preds_flag     = True
load_models_flag    = True
predict_only_hi_z   = False
use_zeroth_model    = False
use_second_z_model  = False  # z >= 3.6 (with SMOGN), or, if needed, z >= 4.0

In [4]:
used_area           = 'S82'  # can be 'S82', 'HETDEX', 'COSMOS'

In [5]:
file_name_dict      = {'S82': gv.file_S82, 'HETDEX': gv.file_HETDEX, 'COSMOS': gv.file_COSMOS}
file_name           = file_name_dict[used_area]

In [6]:
feats_2_disc_S82    = ['objID', 'RA_MILLI', 'DEC_MILLI', 'X_ray_detect', 'W1mag', 'W2mag', 'FUVmag', 'NUVmag',\
                       'num_imputed', 'radio_detect', 'TGSS_detect', 'VLASS_detect']
feats_2_disc_HETDEX = ['objID', 'RA_MILLI', 'DEC_MILLI', 'X_ray_detect', 'W1mag', 'W2mag', 'FUVmag', 'NUVmag',\
                       'num_imputed', 'radio_detect', 'TGSS_detect', 'VLASS_detect', 'LoLSS_detect']
feats_2_disc_COSMOS = ['objID', 'RA_MILLI', 'DEC_MILLI', 'X_ray_detect', 'W1mag', 'W2mag', 'FUVmag', 'NUVmag',\
                       'num_imputed', 'radio_detect', 'TGSS_detect', 'VLASS_detect']

feats_2_disc        = {'S82': feats_2_disc_S82, 'HETDEX': feats_2_disc_HETDEX, 'COSMOS': feats_2_disc_COSMOS}
features_2_discard  = feats_2_disc[used_area]

In [7]:
full_catalog_df     = pd.read_hdf(gv.cat_path + file_name, key='df').drop(columns=features_2_discard)

In [8]:
if used_area == 'S82':
    full_catalog_df.loc[:, 'LOFAR_detect'] = full_catalog_df.loc[:, 'VLAS82_detect'].copy()
    full_catalog_df = full_catalog_df.drop(columns=['VLAS82_detect'])
if used_area == 'COSMOS':
    full_catalog_df.loc[:, 'LOFAR_detect'] = full_catalog_df.loc[:, 'COSMOSVLA3_detect'].copy()
    full_catalog_df = full_catalog_df.drop(columns=['COSMOSVLA3_detect'])

Create features with class and combined redshift.

In [9]:
full_catalog_df['class']            = full_catalog_df.loc[:, 'is_AGN'].copy()
filter_non_confirmed                = np.array(full_catalog_df.loc[:, 'is_AGN'] == 1) | np.array(full_catalog_df.loc[:, 'is_gal'] == 1)
full_catalog_df.loc[~filter_non_confirmed, 'class'] = np.nan
idx_non_Z                           = full_catalog_df.loc[:, 'Z'].where(full_catalog_df.loc[:, 'Z'] > 0).isna()
full_catalog_df.loc[idx_non_Z, 'Z'] = full_catalog_df.loc[:, 'Z'].mask(idx_non_Z, full_catalog_df.loc[idx_non_Z, 'zsp'])

Create column for detection as Radio AGN

In [10]:
full_catalog_df['radio_AGN']      = (np.array(full_catalog_df.loc[:, 'is_AGN'] == 1) & np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1)).astype(int)

Discard minor features.

In [11]:
full_catalog_df                     = full_catalog_df.drop(columns=['is_AGN', 'is_SDSS_QSO', 'is_SDSS_gal', 'is_gal', 'zsp'])

#### As we want to predict, only use sources that have not previous spectroscopic classification

In [12]:
print(f'The used data set (in {used_area}) has {len(full_catalog_df):,} sources.')
print(f'And {np.sum(filter_non_confirmed):,} have previous spectroscopic classification.')

The used data set (in S82) has 369,093 sources.
And 3,304 have previous spectroscopic classification.


In [13]:
full_catalog_df                     = full_catalog_df.loc[~filter_non_confirmed]

In [14]:
print(f'This pipeline will predict properties for {len(full_catalog_df):,} sources in {used_area}.')

This pipeline will predict properties for 365,789 sources in S82.


---

### Load models

In [15]:
if load_models_flag:
    if use_zeroth_model:
        star_clf        = pyc.load_model(gv.models_path + gv.star_model)  # star/no-star model
        cal_star_clf    = load(gv.models_path + gv.cal_str_model)  # calibrated model
    AGN_gal_clf         = pyc.load_model(gv.models_path + gv.AGN_gal_model)  #
    cal_AGN_gal_clf     = load(gv.models_path + gv.cal_AGN_gal_model)  # calibrated model
    radio_det_clf       = pyc.load_model(gv.models_path + gv.radio_model)  # without predicted AGN
    cal_radio_det_clf   = load(gv.models_path + gv.cal_radio_model)  # calibrated model
    redshift_reg        = pyr.load_model(gv.models_path + gv.full_z_model)  # to use on full sample
    if use_second_z_model:
        redshift_reg_2  = pyr.load_model(gv.models_path + gv.high_z_model)  # sources with predicted z >= 3.6

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [16]:
full_catalog_df = pyc.predict_model(AGN_gal_clf, data=full_catalog_df, probability_threshold=gv.AGN_thresh, raw_score=True, round=10)
full_catalog_df = full_catalog_df.drop(columns=['Score_0'])
full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_class', 'Score_1': 'Score_AGN'})
full_catalog_df['Score_AGN'] = np.around(full_catalog_df.loc[:, 'Score_AGN'], decimals=7)
pred_probs_AGN  = cal_AGN_gal_clf.predict(full_catalog_df.loc[:, 'Score_AGN'])
full_catalog_df['Prob_AGN'] = pred_probs_AGN

In [17]:
full_catalog_df = pyc.predict_model(radio_det_clf, data=full_catalog_df, probability_threshold=gv.radio_thresh, raw_score=True, round=10)
full_catalog_df = full_catalog_df.drop(columns=['Score_0'])
full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_radio', 'Score_1': 'Score_radio'})
full_catalog_df['Score_radio'] = np.around(full_catalog_df.loc[:, 'Score_radio'], decimals=7)
pred_probs_rad  = cal_radio_det_clf.predict(full_catalog_df.loc[:, 'Score_radio'])
full_catalog_df['Prob_radio'] = pred_probs_rad

In [18]:
full_catalog_df = pyr.predict_model(redshift_reg, data=full_catalog_df, round=8)
full_catalog_df = full_catalog_df.rename(columns={'Label': 'pred_Z'})
full_catalog_df['pred_Z'] = np.around(full_catalog_df.loc[:, 'pred_Z'], decimals=3)

In [19]:
redshift_tol    = 0.0
if use_second_z_model:
    full_catalog_df = pyr.predict_model(redshift_reg_2, data=full_catalog_df, round=6)
    filter_pred_z   = full_catalog_df.loc[:, 'pred_Z'] >= (gv.high_z_limit + redshift_tol)
    full_catalog_df.loc[:, 'pred_Z'] = full_catalog_df.loc[:, 'pred_Z'].mask(filter_pred_z, full_catalog_df.loc[filter_pred_z, 'Label'])
    full_catalog_df = full_catalog_df.drop(columns=['Label'])
    full_catalog_df.loc[:, 'pred_Z'] = np.around(full_catalog_df.loc[:, 'pred_Z'], decimals=3)

In [20]:
full_catalog_df['pred_prob_class']    = (full_catalog_df.loc[:, 'Prob_AGN']   >= gv.cal_AGN_thresh).astype(int)
full_catalog_df['pred_prob_radio']    = (full_catalog_df.loc[:, 'Prob_radio'] >= gv.cal_radio_thresh).astype(int)

In [21]:
full_catalog_df['pred_radio_AGN']     = (np.array(full_catalog_df.loc[:, 'pred_class'] == 1) & np.array(full_catalog_df.loc[:, 'pred_radio'] == 1)).astype(int)
full_catalog_df['Score_rAGN']         = full_catalog_df.loc[:, 'Score_AGN'] * full_catalog_df.loc[:, 'Score_radio']
full_catalog_df['pred_prob_rAGN']     = (np.array(full_catalog_df.loc[:, 'pred_prob_class'] == 1) & np.array(full_catalog_df.loc[:, 'pred_prob_radio'] == 1)).astype(int)
full_catalog_df['Prob_rAGN']          = full_catalog_df.loc[:, 'Prob_AGN'] * full_catalog_df.loc[:, 'Prob_radio']

rad_score_scaler                      = MinMaxScaler()
full_catalog_df['scaled_score_radio'] = rad_score_scaler.fit_transform(full_catalog_df.loc[:, 'Score_radio'].values.reshape(-1, 1))
full_catalog_df['scaled_score_rAGN']  = full_catalog_df.loc[:, 'Score_AGN'] * full_catalog_df.loc[:, 'scaled_score_radio']

In [22]:
full_catalog_df.loc[:, ['class', 'pred_class', 'LOFAR_detect', 'pred_radio', 'Z', 'pred_Z']].describe()

,class,pred_class,LOFAR_detect,pred_radio,Z,pred_Z
count,0.0,365789.000000,365789.000000,365789.000000,210.000000,365789.000000
mean,NaN,0.123779,0.039222,0.096925,0.684951,2.197085
std,NaN,0.329330,0.194123,0.295856,0.833093,0.813032
min,NaN,0.000000,0.000000,0.000000,-0.002420,0.025000
25%,NaN,0.000000,0.000000,0.000000,0.000120,2.010000
50%,NaN,0.000000,0.000000,0.000000,0.400000,2.262000
75%,NaN,0.000000,0.000000,0.000000,1.100000,2.345000
max,NaN,1.000000,1.000000,1.000000,4.300000,4.707000


Obtain intermediate metrics

In [23]:
filter_radio_AGN_t      = np.array(full_catalog_df.loc[:, 'class'] == 1) & np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1)
filter_gal_AGN_t        = np.array(full_catalog_df.loc[:, 'class'] == 0) | np.array(full_catalog_df.loc[:, 'class'] == 1)
total_size              = len(full_catalog_df)
filter_AGN_t            = np.array(full_catalog_df.loc[:, 'class'] == 1)
num_AGN_t               = np.sum(filter_AGN_t)
num_gal_t               = np.sum(np.array(full_catalog_df.loc[:, 'class'] == 0))
num_radio_t             = np.sum(np.array(full_catalog_df.loc[:, 'LOFAR_detect'] == 1))
num_radio_AGN_t         = np.sum(filter_radio_AGN_t)

In [24]:
# filter_radio_AGN_p      = np.array(full_catalog_df.loc[:, 'Score_AGN']          >= threshold_AGN) &\
#                             np.array(full_catalog_df.loc[:, 'Score_radio']      >= threshold_radio)
filter_radio_AGN_p      = np.array(full_catalog_df.loc[:, 'pred_class']      == 1) &\
                            np.array(full_catalog_df.loc[:, 'pred_radio']    == 1)
filt_hiz_rAGN_p         = filter_radio_AGN_p * np.array(full_catalog_df.loc[:, 'pred_Z'] >= gv.high_z_limit)
filter_AGN_p            = np.array(full_catalog_df.loc[:, 'pred_class']      == 1)
filter_radio_p          = np.array(full_catalog_df.loc[:, 'pred_radio']      == 1)
num_AGN_p               = np.sum(filter_AGN_p)
num_gal_p               = np.sum(np.array(full_catalog_df.loc[:, 'pred_class'] == 0))
num_radio_p             = np.sum(filter_radio_p)
num_radio_AGN_p         = np.sum(filter_radio_AGN_p)

### Select sources predicted to be Radio AGN

In [25]:
full_catalog_df         = full_catalog_df.loc[filter_radio_AGN_p]

In [26]:
full_catalog_df.loc[:, ['class', 'pred_class', 'LOFAR_detect', 'pred_radio', 'Z', 'pred_Z']].describe()

,class,pred_class,LOFAR_detect,pred_radio,Z,pred_Z
count,0.0,5569.0,5569.000000,5569.0,53.000000,5569.000000
mean,NaN,1.0,0.136470,1.0,0.721386,1.835062
std,NaN,0.0,0.343318,0.0,0.721763,0.903158
min,NaN,1.0,0.000000,1.0,-0.000170,0.025000
25%,NaN,1.0,0.000000,1.0,0.300000,1.023000
50%,NaN,1.0,0.000000,1.0,0.500000,2.081000
75%,NaN,1.0,0.000000,1.0,1.000000,2.515000
max,NaN,1.0,1.000000,1.0,3.100000,4.567000


Add individual metrics for redshift

In [27]:
full_catalog_df['Delta_z_N'] = np.around((full_catalog_df.loc[:, 'pred_Z'] - full_catalog_df.loc[:, 'Z']) /\
                            (1 + full_catalog_df.loc[:, 'Z']), decimals=3)

full_catalog_df['sigma_NMAD'] = np.around(1.48 * np.abs(full_catalog_df.loc[:, 'pred_Z'] - full_catalog_df.loc[:, 'Z']) /\
                            (1 + full_catalog_df.loc[:, 'Z']), decimals=3)

Numerical summary.

In [28]:
str_0_t   = f'Out of {total_size:,} initial sources in {used_area},\n'
str_1_t   = f'{num_gal_t:,} are confirmed to be galaxies. On the other side,\n'
str_2_t   = f'{num_AGN_t:,} are confirmed to be AGN. And, from the AGN,\n'
str_3_t   = f'{num_radio_AGN_t:,} are detected in radio.'

str_0_p   = f'Out of {num_radio_AGN_t:,} initial radio-detected AGN in {used_area},\n'
str_1_p   = f'{num_gal_p:,} are predicted to be galaxies. On the other side,\n'
str_2_p   = f'{num_AGN_p:,} are predicted to be AGN. And, from the predicted AGN,\n'
str_3_p   = f'{num_radio_AGN_p:,} are predicted to be detected in radio.'

In [29]:
print('-' * 60)
print(str_0_t + str_1_t + str_2_t + str_3_t)
print('-' * 60)
print(str_0_p + str_1_p + str_2_p + str_3_p)
print('-' * 60)

------------------------------------------------------------
Out of 365,789 initial sources in S82,
0 are confirmed to be galaxies. On the other side,
0 are confirmed to be AGN. And, from the AGN,
0 are detected in radio.
------------------------------------------------------------
Out of 0 initial radio-detected AGN in S82,
320,512 are predicted to be galaxies. On the other side,
45,277 are predicted to be AGN. And, from the predicted AGN,
5,569 are predicted to be detected in radio.
------------------------------------------------------------


In [30]:
cols_4_table = show_columns = ['Name', 'RA_ICRS', 'DE_ICRS', 'TYPE', 'band_num', 'class', 'pred_class',\
                               'pred_prob_class', 'Score_AGN', 'Prob_AGN', 'LOFAR_detect', 'pred_radio',\
                               'pred_prob_radio', 'Score_radio', 'Prob_radio', 'radio_AGN', 'pred_radio_AGN',\
                               'pred_prob_rAGN', 'Score_rAGN', 'Prob_rAGN', 'Z', 'pred_Z']

In [31]:
cols_4_export_S82    = ['Total_flux_VLASS', 'Fint_VLAS82', 'Stotal_TGSS', 'FEP']
cols_4_export_HETDEX = ['Total_flux_VLASS', 'Sint_LOFAR', 'Stotal_TGSS', 'FEP', 'TotalFlux_LoLSS']
cols_4_export_COSMOS = ['Total_flux_VLASS', 'Stotal_TGSS', 'FEP', 'Flux_COSMOSVLA3']

cols_4_exp_all       = {'S82': cols_4_export_S82, 'HETDEX': cols_4_export_HETDEX, 'COSMOS': cols_4_export_COSMOS}

cols_photo           = ['W1mproPM', 'W2mproPM', 'gmag', 'rmag', 'imag', 'zmag', 'ymag',\
                        'Jmag', 'Hmag', 'Kmag', 'W3mag', 'W4mag']

cols_4_export        = cols_4_table + cols_4_exp_all[used_area] + cols_photo

In [32]:
filter_new_rAGN = full_catalog_df.loc[:, 'radio_AGN'] == 0
if predict_only_hi_z:
    filter_high_z   = full_catalog_df.loc[:, 'pred_Z']    >= gv.high_z_limit
elif not predict_only_hi_z:
    filter_high_z   = np.ones_like(full_catalog_df.loc[:, 'pred_Z']).astype(bool)

In [33]:
full_catalog_df.loc[:, 'TYPE'] = full_catalog_df.loc[:, 'TYPE'].mask(full_catalog_df.loc[:, 'TYPE'] == '    ', 'CCCC')

In [34]:
display(full_catalog_df.loc[filter_high_z, cols_4_export].sort_values(by=['pred_Z'], ascending=False).head(15))

,Name,RA_ICRS,DE_ICRS,TYPE,band_num,class,pred_class,pred_prob_class,Score_AGN,Prob_AGN,LOFAR_detect,pred_radio,pred_prob_radio,Score_radio,Prob_radio,radio_AGN,pred_radio_AGN,pred_prob_rAGN,Score_rAGN,Prob_rAGN,Z,pred_Z,Total_flux_VLASS,Fint_VLAS82,Stotal_TGSS,FEP,W1mproPM,W2mproPM,gmag,rmag,imag,zmag,ymag,Jmag,Hmag,Kmag,W3mag,W4mag
189511,J224122.14+000146.9,340.342285,0.029702,CCCC,8,NaN,1,1,0.996063,0.993517,0,1,1,0.501565,0.679283,0,1,1,0.499590,0.674880,NaN,4.567,15.21,17.860001,11.18,57.900002,18.858999,19.153999,23.299999,21.108000,19.775801,19.137800,18.857500,17.450001,17.24,16.59,16.67,14.62
115779,J222430.17+002812.4,336.125732,0.470109,CCCC,8,NaN,1,1,0.970989,0.963052,0,1,1,0.501565,0.679283,0,1,1,0.487014,0.654185,NaN,4.564,15.21,17.860001,11.18,57.900002,18.687000,19.076000,23.299999,21.013700,19.659700,18.989700,18.677200,17.450001,17.24,16.59,16.67,14.62
177651,J224207.04+002930.9,340.529358,0.491929,CCCC,8,NaN,1,1,0.954704,0.945525,0,1,1,0.501565,0.679283,0,1,1,0.478846,0.642280,NaN,4.564,15.21,17.860001,11.18,57.900002,18.833000,19.196001,23.299999,21.183800,19.565701,18.840099,18.553400,17.450001,17.24,16.59,16.67,14.62
197450,J225613.56+001103.2,344.056549,0.184245,CCCC,8,NaN,1,1,0.943564,0.934006,0,1,1,0.501565,0.679283,0,1,1,0.473259,0.634455,NaN,4.564,15.21,17.860001,11.18,57.900002,18.744999,19.134001,23.299999,20.999901,19.444099,18.729401,18.433800,17.450001,17.24,16.59,16.67,14.62
333861,J004706.74+000528.9,11.778095,0.091367,CCCC,8,NaN,1,1,0.960243,0.951381,0,1,1,0.499585,0.495449,0,1,1,0.479723,0.471361,NaN,4.561,15.21,17.860001,11.18,57.900002,19.128000,19.267000,23.299999,21.629400,20.156700,19.481600,19.170099,17.450001,17.24,16.59,16.67,14.62
62613,J221225.34+011324.1,333.105591,1.223353,CCCC,8,NaN,1,1,0.729516,0.734764,0,1,1,0.499585,0.495449,0,1,1,0.364455,0.364038,NaN,4.554,15.21,17.860001,11.18,57.900002,19.142000,19.400002,23.299999,21.380600,20.080799,19.485500,19.121401,17.450001,17.24,16.59,16.67,14.62
365788,J220524.45+000628.5,331.351929,0.107907,CCCC,8,NaN,1,1,0.815927,0.812911,0,1,1,0.501399,0.665119,0,1,1,0.409105,0.540682,NaN,4.554,15.21,17.860001,11.18,57.900002,18.824999,19.203001,23.299999,21.321100,19.592699,18.818701,18.500099,17.450001,17.24,16.59,16.67,14.62
249207,J010546.22+004001.0,16.442604,0.666933,CCCC,6,NaN,1,1,0.900277,0.891356,0,1,1,0.499585,0.495449,0,1,1,0.449765,0.441622,NaN,4.545,15.21,17.860001,11.18,57.900002,18.973999,19.322001,23.299999,21.435400,19.864201,19.166901,18.824100,17.450001,17.24,16.59,16.67,14.62
312369,J004708.92+010905.2,11.787194,1.151465,CCCC,8,NaN,1,0,0.483044,0.508490,0,1,1,0.499419,0.479379,0,1,0,0.241241,0.243759,NaN,4.543,15.21,17.860001,11.18,57.900002,18.980000,19.302999,23.299999,21.528400,19.806801,18.964800,18.564899,17.450001,17.24,16.59,16.67,14.62
90563,J223402.14+002739.7,338.508972,0.461050,CCCC,8,NaN,1,1,0.953158,0.943908,0,1,1,0.499666,0.503301,0,1,1,0.476260,0.475070,NaN,4.543,15.21,17.860001,11.18,57.900002,18.901999,19.103001,23.299999,21.537100,20.072100,19.310400,18.936600,17.450001,17.24,16.59,16.67,14.62


In [35]:
if save_preds_flag:
    full_catalog_df.loc[filter_high_z, cols_4_export].sort_values(by=['pred_Z'],
     ascending=False).to_csv(gv.preds_path + f'predicted_rAGN_{used_area}.csv')

---